In [1]:
# Installing required packages
%pip install -r requirements.txt
%pip install --no-index --find-links=wheelhouse heapcy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: wheelhouse
Note: you may need to restart the kernel to use updated packages.


In [2]:
size_of_heap = 100_000
import random
def gen_random_string(le=12)->str:
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789~!@#$%^&*-_=+;:\\`"
    random.seed()
    pwd_len= random.randrange(1,le)
    prob = random.uniform(0,1)
    final_str= ""
    for _ in range(pwd_len):
        char= random.randrange(le)
        final_str += alphabet[char]
    return final_str + " "+str(prob)

In [3]:
import tempfile
def create_temp_file_for_benchmark()->str:
    tf = tempfile.NamedTemporaryFile(delete=False)
    lis = [ gen_random_string() for _ in range(size_of_heap)]
    with open(tf,"w") as f_open:
        f_open.writelines(lis)
    return tf.name
    

In [4]:
from pympler import asizeof

In [5]:
from heapitem import HeapItem
import heapq
heap = []
for i in range(size_of_heap):
    heapq.heappush(heap,HeapItem(0.5555,gen_random_string()))
    
print(f"sizeof normal heap with heapitem = {asizeof(heap)}")
del heap


TypeError: 'module' object is not callable

In [ ]:
import heapcy,os
heap = heapcy.Heap(size_of_heap)
tf = create_temp_file_for_benchmark()
with open(tf,"rb") as f_open:

    while True:
        offset = f_open.tell()
        line = f_open.readline()
        if not line:
            break

        parts = line.rstrip(b"\b\n").split(b" ", 1)
        if len(parts) != 2:
            continue

        prob = float(parts[1].decode(encoding="ascii"))
        if len(heap)< size_of_heap:
            heapcy.heappush(heap)
        elif heapcy.heappeek(heap)[0]>prob:
            heapcy.heappoppush(heap,prob,offset)

print(f"sizeof special heap = {asizeof(heap)}")
del heap



In [ ]:
import heapq 
heap = []

with open(tf,"rb") as f_open:
    for line in f_open.readline():

        parts = line.split(" ")
        if len(parts) != 2:
            continue

        prob = float(parts[1].decode(encoding="ascii"))

        if len(heap)< size_of_heap:
            heapq.heappush(heap,(prob,parts[0]))
        elif heap[0][0]>prob:
            heapq.heappoppush(heap,prob,offset)

print(f"sizeof special heap = {asizeof(heap)}")
del heap
os.remove(tf)